In [2]:
from two_step_classification import *
print_version()

ran:  2023-09-14 09:08:44.740982
python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [3]:
results: list[dict[str, ]] = []

In [4]:
# load data
data_paths = ["models/kdd99_features/x_test_df.pkl",
              "models/kdd99_features/x_test-drop_25_df.pkl",
              "models/kdd99_features/x_test-drop+ae_30_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test-drop+ae_40_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test+ae_43_df&activation=relu&epochs=5&batch_size=32.pkl",
              "models/kdd99_features/x_test+ae_48_df&activation=relu&epochs=5&batch_size=32.pkl",
              ]
lgb_binary_paths = ["models/lightgbm/lgb_dropped_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped_25_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_30_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_35_binary_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_40_binary_tuned_booster.model",
                    "models/lightgbm/lgb+ae_43_binary_tuned_booster.model",
                    "models/lightgbm/lgb+ae_48_binary_tuned_booster.model",
                    ]
lgb_anomaly_paths = ["models/lightgbm/lgb_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped_25_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_30_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_35_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb_dropped+ae_40_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb+ae_43_mapped_anomaly_tuned_booster.model",
                    "models/lightgbm/lgb+ae_48_mapped_anomaly_tuned_booster.model",]



In [5]:
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)
y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")

def predict(x_path, path_1st, path_2nd):
    x_test = pd.read_pickle(x_path)
    model_1st: lgb.Booster = lgb.Booster(model_file=path_1st)
    model_2nd: lgb.Booster = lgb.Booster(model_file=path_2nd)
    y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
    predicted_indexes = y_pred_binary[y_pred_binary == 1].index
    x_anomalies = x_test.loc[predicted_indexes]
    y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
    y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
    y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
    y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
    
    y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)
    
    cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])
    
    cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())
    
    print(classification_report(y_test.sort_index(), y_pred.sort_index()))